In [2]:
from database_functions import *
from database_utils import *

In [3]:
query = timebucket_values_from_database_sql('swiss_landschlacht_63', '2023-01-25 02:30:00', '2023-02-08 03:30:00', timebucket='5m', agg_function='MAX', return_query=True)

In [4]:
def run_query(query):
    with psycopg2.connect(CONNECTION) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        tuple_list = cursor.fetchall()
        return tuple_list

In [118]:
def run_query(query, params=None):
    with psycopg2.connect(CONNECTION) as conn:
        with conn.cursor() as cursor:
            if params:
                cursor.execute(query, params)
            else:
                cursor.execute(query)
            if cursor.description:
                return cursor.fetchall()
            else:
                return None

In [219]:
def commit_query(query):
    with psycopg2.connect(CONNECTION) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query)
            conn.commit()
    return  

In [234]:
commit_query('DROP FUNCTION IF EXISTS get_column_aggregation(text);')

In [261]:
query = """
CREATE OR REPLACE FUNCTION column_aggregation_query(p_table_name text, p_start_time timestamp, p_end_time timestamp, p_timebucket text, p_agg_function text)
RETURNS TABLE (time timestamp, column_name text, value numeric) AS $$
DECLARE
  columns text[];
  col text;
  query text;
BEGIN
  SELECT ARRAY_AGG(column_name::text) INTO columns
  FROM information_schema.columns
  WHERE table_name = p_table_name
    AND column_name <> 'datetime';

  query := 'SELECT time_bucket($1, datetime) AS time, ';
  FOREACH col IN ARRAY columns LOOP
    query := query || 'AVG("' || col || '") AS "' || col || '", ';
  END LOOP;
  query := LEFT(query, LENGTH(query) - 2) || ' FROM ' || p_table_name ||
           ' WHERE datetime BETWEEN $2 AND $3 ' ||
           ' GROUP BY time ';
  query := query || ' ORDER BY time';

  EXECUTE query USING p_timebucket, p_start_time, p_end_time;
END;
$$ LANGUAGE plpgsql;
"""

In [262]:
query = """
CREATE OR REPLACE FUNCTION column_aggregation_query(p_table_name text, p_start_time timestamp, p_end_time timestamp, p_timebucket text, p_agg_function text)
RETURNS text AS $$
DECLARE
  columns text[];
  col text;
  query_text text;
BEGIN
  SELECT ARRAY_AGG(column_name::text) INTO columns
  FROM information_schema.columns
  WHERE table_name = p_table_name
    AND column_name <> 'datetime';

  query_text := 'SELECT time_bucket($1, datetime) AS time, ';
  FOREACH col IN ARRAY columns LOOP
    query_text := query_text || p_agg_function'("' || col || '") AS "' || col || '", ';
  END LOOP;
  query_text := LEFT(query_text, LENGTH(query_text) - 2) || ' FROM ' || p_table_name ||
           ' WHERE datetime BETWEEN $2 AND $3 ' ||
           ' GROUP BY time ';
  query_text := query_text || ' ORDER BY time';

  RETURN query_text;
END;
$$ LANGUAGE plpgsql;
"""
 



In [259]:
commit_query(query)

In [263]:
run_query("SELECT column_aggregation_query('swiss_landschlacht_63', '2023-01-25 02:30:00', '2023-02-08 03:30:00', '5m', 'AVG');")

UndefinedObject: type "p_agg_function" does not exist
LINE 1: query_text := query_text || p_agg_function'("' || col || '")...
                                    ^
QUERY:  query_text := query_text || p_agg_function'("' || col || '") AS "' || col || '", '
CONTEXT:  PL/pgSQL function column_aggregation_query(text,timestamp without time zone,timestamp without time zone,text,text) line 14 at assignment


In [39]:
def get_aggregate_columns(table_name):
    query = f'''
        SELECT STRING_AGG('AVG("'|| column_name || '") AS "' || column_name || '"', ', ' ORDER BY column_name::numeric)
        FROM information_schema.columns
        WHERE table_name = '{table_name}'
          AND column_name NOT IN ('datetime');
    '''
    return run_query(query)[0][0]


In [40]:
get_aggregate_columns('swiss_landschlacht_63')

'AVG("45.0") AS "45.0", AVG("45.5") AS "45.5", AVG("46.375") AS "46.375", AVG("46.5629997253418") AS "46.5629997253418", AVG("47.375") AS "47.375", AVG("47.6879997253418") AS "47.6879997253418", AVG("48.375") AS "48.375", AVG("48.5629997253418") AS "48.5629997253418", AVG("49.25") AS "49.25", AVG("49.5629997253418") AS "49.5629997253418", AVG("50.4379997253418") AS "50.4379997253418", AVG("50.6879997253418") AS "50.6879997253418", AVG("51.3129997253418") AS "51.3129997253418", AVG("51.6879997253418") AS "51.6879997253418", AVG("52.3129997253418") AS "52.3129997253418", AVG("52.6879997253418") AS "52.6879997253418", AVG("53.375") AS "53.375", AVG("53.5629997253418") AS "53.5629997253418", AVG("54.375") AS "54.375", AVG("54.625") AS "54.625", AVG("55.4379997253418") AS "55.4379997253418", AVG("55.8129997253418") AS "55.8129997253418", AVG("56.4379997253418") AS "56.4379997253418", AVG("56.5") AS "56.5", AVG("57.375") AS "57.375", AVG("57.75") AS "57.75", AVG("58.4379997253418") AS "58.43

In [1]:
get_aggregate_columns('swiss_landschlacht_63')

NameError: name 'get_aggregate_columns' is not defined

In [73]:
run_query(query)

ProgrammingError: no results to fetch

In [76]:
query = """SELECT * FROM get_aggregate_columns('swiss_landschlacht_63');"""

In [77]:
run_query(query)

AmbiguousColumn: column reference "table_name" is ambiguous
LINE 19:         WHERE table_name = 'swiss_landschlacht_63'
                       ^
DETAIL:  It could refer to either a PL/pgSQL variable or a table column.
QUERY:  SELECT STRING_AGG(agg_function || '(' || column_name || ')', ', ')
                    FROM (
        SELECT column_name,
               CASE
                   WHEN data_type = 'integer' THEN 'avg'
                   WHEN data_type = 'numeric' THEN 'avg'
                   WHEN data_type = 'double precision' THEN 'avg'
                   WHEN data_type = 'bigint' THEN 'avg'
                   WHEN data_type = 'smallint' THEN 'avg'
                   WHEN data_type = 'real' THEN 'avg'
                   WHEN data_type = 'timestamp without time zone' THEN 'avg'
                   WHEN data_type = 'timestamp with time zone' THEN 'avg'
                   WHEN data_type = 'date' THEN 'avg'
                   WHEN data_type = 'time without time zone' THEN 'avg'
       

In [5]:
get_table_names_sql()

['malaysia_banting_01',
 'india_ooty_02',
 'ssrt_59',
 'roswell_nm_57',
 'poland_grotniki_03',
 'egypt_alexandria_01',
 'usa_arizona_erau_01',
 'spain_siguenza_02',
 'mexico_lance_a_63',
 'germany_dlr_62',
 'mexico_lance_b_62',
 'egypt_alexandria_02',
 'australia_lmro_59',
 'india_gauri_59',
 'australia_assa_60',
 'romania_01',
 'almaty_58',
 'swiss_hb9sct_01',
 'bir_01',
 'alaska_haarp_62',
 'glasgow_01',
 'mrt1_02',
 'alaska_cohoe_63',
 'alaska_cohoe_62',
 'arecibo_observatory_62',
 'australia_assa_63',
 'norway_ny_aalesund_01',
 'germany_dlr_63',
 'humain_59',
 'germany_dlr_01',
 'india_ooty_01',
 'spain_peralejos_03',
 'mongolia_ub_01',
 'austria_oe3flb_55',
 'triest_60',
 'swiss_irsol_01',
 'spain_peralejos_62',
 'mro_62',
 'alaska_haarp_63',
 'roswell_nm_59',
 'triest_59',
 'spain_peralejos_63',
 'algeria_craag_59',
 'japan_ibaraki_59',
 'swiss_muhen_62',
 'mrt3_03',
 'norway_egersund_01',
 'roswell_nm_58',
 'mro_59',
 'austria_oe3flb_57',
 'alaska_anchorage_01',
 'hurbanovo_59',